# 75.06 / 95.58 Organización de Datos
# Trabajo Práctico 1

### Grupo Datatouille

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

> https://github.com/fdelmazo/7506-datos

In [ ]:
# Set-Up inicial

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import squarify # https://github.com/laserson/squarify ( pip install squarify )
import geopandas as gpd # http://geopandas.org/install.html ( conda install -c conda-forge geopandas o pip install geopandas )
from shapely.geometry import Point
from wordcloud import WordCloud # pip install wordcloud

sns.set(style="darkgrid")
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['figure.figsize'] = (15,5)

df = pd.read_csv('events.csv', low_memory=False, index_col='timestamp')

## Información general del dataset

In [ ]:
with pd.option_context('display.max_column',0):
    display(df.sample(n=5))

In [ ]:
print('{} atributos y {} registros en el dataframe.\n'.format(df.shape[1],df.shape[0]))
print('Primer registro: {} \nÚltimo registro: {}.'.format(df.index.min(),df.index.max()))

In [ ]:
bytes_used = df.memory_usage().sum()
df.info()

In [ ]:
with pd.option_context('display.max_colwidth',-1):
    display(pd.read_csv('columns-desc.csv',index_col='column'))

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

---
#### Pasado el vistazo general a los datos, se pueden transformar datos en sus tipos correspondientes para mejor manejo de estos y para ahorro de memoria

In [ ]:
# Los atributos con pocos valores posibles se pasan a variables categoricas para ahorrar memoria
df['event'] = df['event'].astype('category')
df['condition'] = df['condition'].astype('category')
df['storage'] = df['storage'].astype('category')
df['search_engine'] = df['search_engine'].astype('category')
df['channel'] = df['channel'].astype('category')
df['device_type'] = df['device_type'].astype('category')

# El tiempo es mejor manejarlo como tal
df.index = pd.to_datetime(df.index)

# Teniendo solamente dos valores posibles para el tipo de usuario, y priorizando los nuevos, se transforma a una variable booleana
df = df.rename(columns={'new_vs_returning':'new_user'})
d = {'New':True, 'Returning':False}
df['new_user'].map(d)
df['new_user'] = df['new_user'].astype('bool')

In [ ]:
# Chequeo
df.info()

ahorro = (bytes_used - df.memory_usage().sum()) /1000000
print("Memoria ahorrada: {:.4f}MB".format(ahorro))

---
## Información de eventos

In [ ]:
event = df['event']
event.describe()

In [ ]:
with pd.option_context('display.max_colwidth',0):
    display(pd.read_csv('events-desc.csv',index_col='event'))

In [ ]:
#data = event.value_counts().head(7)
#ax = sns.barplot(x=data.index,y=data)
#ax.set(xlabel='Count',ylabel='Evento')

# Mismo grafico, de otra forma (tarda más tiempo (escribir %%time al comienzo de la celda))
orden = event.value_counts().head(7).index
visu = sns.countplot(x='event',data=df,order=orden)
visu.axes.set_title('Cantidad de visitas segun evento')

---
## Información geográfica

In [ ]:
countries = df['country'].value_counts()
countries = countries.drop('Unknown')
data = countries.head(3)

visu = squarify.plot(data, label=data.index, alpha=.5, color=['green','red','cyan'])
visu.axes.set_title('Países con más visitas')

In [ ]:
# Siempre supimos (asumimos) que Brazil sería el país con mayor tráfico. 
# Este dato no nos deja ver la proporción del resto de los países
countries = df['country'].value_counts()
countries = countries.drop('Unknown')
countries = countries.drop('Brazil')
data = countries.head(7)

visu = squarify.plot(data, label=data.index, alpha=.5, color=['red','cyan','yellow','grey','purple','orange','blue'])
visu.axes.set_title('Países con más visitas, exceptuando Brazil')

### Sacando las longitudes y latitudes de distintas ciudades del mundo, podemos ver que zonas de los 4 países con más trafico son las que más visitas tienen. 

Con ayuda del módulo geopandas, podemos plotear directamente sobre un mapa
Las bases de datos adicionales fueron sacadas de http://www.geonames.org/

In [ ]:
BR = pd.read_csv('BR.csv', low_memory=False, sep='\t')
BR = BR[['name','latitude','longitude']]

# Para que geopandas pueda leer bien las latitudes y longitudes, deben ser pasadas al tipo Point
BR['coordinates'] = list(zip(BR['longitude'],BR['latitude']))
BR['coordinates'] = BR['coordinates'].apply(Point)
BR.head()

In [ ]:
# Se debe hacer un join de los datos que se tienen (nombre de ciudad, cantidad de eventos) y los datos de geonames (nombre de ciudad, punto en el mapa), y esto plotearlo sobre los datos de geopandas (nombre de pais, punto en el mapa mundial)

# Se preparan los datos para el join (inner join de nombre de ciudad (columna name))
ciudades_brazil = df.loc[(df['country'] == 'Brazil')]
data = ciudades_brazil['city'].value_counts()
data = data.drop('Unknown')
data = data.to_frame()
data.reset_index(inplace=True)
data = data.rename(columns={'index':'name','city':'count'})

# Se pasa de un dataframe normal de pandas a uno de geopandas
BRA = gpd.GeoDataFrame(BR, geometry='coordinates')

# Se hace el join (en geopandas: merge) de ambos sets. Siendo que geonames daba mucha más información de la necesaria, duplicando valores por ciudades, se borran los duplicados
data = BRA.merge(data, on='name')
data = data.drop_duplicates('name')

# Se prepara el 'fondo' del gráfico, siendo este nomás el país. Para esto se usan los datos por defecto de geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world[world.name=='Brazil'].plot(color='white',edgecolor='black')

visu = data.plot(ax=ax,cmap='OrRd')
visu.axes.set_title('Ciudades de Brazil con más visitas')

In [ ]:
US = pd.read_csv('US.csv', low_memory=False, sep='\t')
US = US[['name','latitude','longitude']]
US['coordinates'] = list(zip(US['longitude'],US['latitude']))
US['coordinates'] = US['coordinates'].apply(Point)

In [ ]:
ciudades_argentina = df.loc[(df['country'] == 'Argentina')]
data = ciudades_argentina['city'].value_counts()
data

# Viendo que la mayoría de los eventos de Argentina vienen de Buenos Aires, hacer el gráfico sobre el mapa sería trivial

In [ ]:
ciudades_usa = df.loc[(df['country'] == 'United States')]
data = ciudades_usa['city'].value_counts()
data = data.drop('Unknown')
data = data.to_frame()
data.reset_index(inplace=True)
data = data.rename(columns={'index':'name','city':'count'})

USA = gpd.GeoDataFrame(US, geometry='coordinates')

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world
ax = world[world.name=='United States'].plot(color='white',edgecolor='black')

data = USA.merge(data, on='name')
data = data.drop_duplicates('name')

visu = data.plot(ax=ax,cmap='Blues')
visu.axes.set_title('Ciudades de Estados Unidos con más visitas')

In [ ]:
st = df['search_term'].dropna()
st = st.apply(lambda x: x.lower())
st = st.value_counts().to_frame()
st = st.loc[(st['search_term'] >= 300)]
st

In [ ]:
# Create a list of word
text = ''
for w,q in zip(st.index,st['search_term']):
    text += ' '.join([w for x in range(q)])

text = ' '.join([s for s in text.split() if len(s)>2])    

wordcloud = WordCloud(width=2000, height=800, margin=0,collocations=False).generate(text)
 
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()


---
## Conclusiones (insights)